<a href="https://colab.research.google.com/github/Shankar-Hadimani/ai-ml-engineering/blob/master/notebook/04_transfer_learning_tensorflow_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import os
import datetime

In [4]:
dataset, info = tfds.load('amazon_us_reviews/Major_Appliances_v1_00', with_info=True, split=('train[:70%]','train[70%:85%]','train[85%:]'))
info

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Major_Appliances_v1_00/0.1.0.incompleteWC4CAD/amazon_us_reviews-train.tfrecord


Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Major_Appliances_v1_00/0.1.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [6]:
train_dataset=dataset[0]
test_dataset=dataset[1]
valid_dataset=dataset[2]

In [7]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'48220985'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Major Appliances'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B002ZD5I16'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'476878110'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Whirlpool Part Number 4388736: DRUM'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'Just was described, works great, fast shipping!!'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2015-04-08'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'Perfect Fit'>, 'review_id': <tf.Tensor: shape=(), dtype=string, numpy=b'R2PIXJFK315V9F'>, 'star_rating': <tf.Tensor: shape=(), dtype=int32, numpy=5>, 'total_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'ver

In [8]:
### decorator '@tf.function' is used to compile the underneath 
## function as callable tensorflow graph

@tf.function
def target(x):
  if x > 3:
    return 1
  else:
    return 0


In [9]:
### visualize some first 5 records as numpy arrays from Tensors
for reviews  in  train_dataset.take(5):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())

  star_rating = review_text.get('star_rating')
  print(star_rating)
  print(target(star_rating).numpy())

b'Just was described, works great, fast shipping!!'
tf.Tensor(5, shape=(), dtype=int32)
1
b'Amazing replacement! I kept buying the exspensive name brand filters and decided to try this out. You could not tell the difference! Awesome product!'
tf.Tensor(5, shape=(), dtype=int32)
1
b"The round rubbers that holds the spin dryer usually break down into pieces that your spin dryer doesn't turned due to balance.. Good product keeping your clothes dry quickly, but bad because of the rubber shies break up.."
tf.Tensor(2, shape=(), dtype=int32)
0
b"Absolutely love my new laundry pod! The only thing that would make me love it more was if there was a left-handed option available. The green grip on the cover is awesome for my right-handed boyfriend; not so much for me. Granted, I could use it backwards, but it's so much cuter front facing!"
tf.Tensor(5, shape=(), dtype=int32)
1
b'We have had this refrigerator for 1 year.  The repairmen have come out 3 times and the ice maker/water dispenser still 

In [10]:
### create function to conevrt the tensors to true values
def fetch(tensor):
  text = tensor['data'].get('review_body')
  label = target(tensor['data'].get('star_rating'))
  return text, label

In [13]:
### apply fetch function on train, test and valudation datasets 
train_data_df = train_dataset.map(fetch)
test_data_df = test_dataset.map(fetch)
valid_dataset_df = valid_dataset.map(fetch)

In [16]:
### view the first 5 labels and data from train_data 
train_data, train_label = next(iter(train_data_df.batch(5)))
train_data 

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Just was described, works great, fast shipping!!',
       b'Amazing replacement! I kept buying the exspensive name brand filters and decided to try this out. You could not tell the difference! Awesome product!',
       b"The round rubbers that holds the spin dryer usually break down into pieces that your spin dryer doesn't turned due to balance.. Good product keeping your clothes dry quickly, but bad because of the rubber shies break up..",
       b"Absolutely love my new laundry pod! The only thing that would make me love it more was if there was a left-handed option available. The green grip on the cover is awesome for my right-handed boyfriend; not so much for me. Granted, I could use it backwards, but it's so much cuter front facing!",
       b'We have had this refrigerator for 1 year.  The repairmen have come out 3 times and the ice maker/water dispenser still is not right.  Water freezes up in the top of the refrigerator; ice 

In [17]:
train_label

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 1, 0, 1, 0], dtype=int32)>

In [31]:
### import tensor flow hub

""" 
we haven't done anything with our raw datasets. It has no tokenisation ,stemming and  stop word removal etc..
Use the embedding layer Google Pre-trained tensor flow model from tensor flow hub, based on Neural Net Language
This model is trained for embeddings for 128 dimensions and was trained on 200 billion corpus on Google News datasets

purpose: It takes data, splits on words based on spaces and finally return individual embedding for each word and finally return a concated 
single sentence tensor 
"""

" \nwe haven't done anything with our raw datasets. It has no tokenisation ,stemming and  stop word removal etc..\nUse the embedding layer Google Pre-trained tensor flow model from tensor flow hub, based on Neural Net Language\nThis model is trained for embeddings for 128 dimensions and was trained on 200 billion corpus on Google News datasets\n\npurpose: It takes data, splits on words based on spaces and finally return individual embedding for each word and finally return a concated \nsingle sentence tensor \n"

In [40]:
embedding = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-9.51282755e-02, -9.18786898e-02, -5.37700355e-02,
         1.08619668e-01,  4.07037511e-02, -5.80801852e-02,
        -5.21476083e-02,  3.19720097e-02,  1.56951353e-01,
        -9.36029926e-02, -5.06434254e-02,  1.50606001e-03,
         6.62063658e-02, -9.65742245e-02, -6.13453984e-02,
         2.53834147e-02,  3.55761275e-02, -9.14261211e-03,
        -5.73733859e-02, -5.07911257e-02,  6.43319190e-02,
        -1.17676198e-01,  6.34124910e-04,  1.81849897e-02,
         7.11547257e-03, -2.71911006e-02,  7.46759325e-02,
        -1.56106636e-01,  1.66973457e-01, -4.39601801e-02,
        -8.42082351e-02, -9.65642333e-02, -6.22393601e-02,
         1.67741507e-01,  3.37394774e-02,  1.20292995e-02,
         3.07388641e-02, -5.44720143e-02, -3.84759828e-02,
         7.27326497e-02,  1.50642246e-01,  1.24470912e-01,
         2.02967450e-01, -1.53443798e-01,  7.29663298e-02,
         8.31599161e-02, -4.33836840e-02, -9.69492719e-02,
      

In [41]:
### add additonal layer to the keras 
## avoid overfit by calling drop out
model = tf.keras.Sequential()
model.add(hub_layer)   ### Dense layer from pretrained model
model.add(tf.keras.layers.Dense(32, activation='relu'))   ## Dense layer with 32 units
model.add(tf.keras.layers.Dropout(0.2))   ### drop to avoid overfit
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  ## classification layer by giving  dense layer with single (1) unit


In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 124,646,849
Trainable params: 124,646,849
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [44]:
"""
adding batch size of 512 rows at a time. 
and train dataset is even shuffled for first 70000 rows
"""
train_data_df=train_data_df.shuffle(70000).batch(512)
valid_dataset_df=valid_dataset_df.batch(512)

In [45]:
history = model.fit(train_data_df,
                    epochs=2,
                    validation_data=valid_dataset_df,
                    verbose=1)

Epoch 1/2
133/133 [==============================] - 186s 1s/step - loss: 0.5482 - accuracy: 0.8071 - val_loss: 0.5032 - val_accuracy: 0.8758
Epoch 2/2
133/133 [==============================] - 183s 1s/step - loss: 0.4921 - accuracy: 0.8950 - val_loss: 0.4957 - val_accuracy: 0.8821


In [48]:
## let match against test adta sets. accuracy after validation should align with test data set as well

"""
Validatin accuracy we got above nearly matches with test datasets accuracy which is about 88% 
"""

results = model.evaluate(test_data_df.batch(512), verbose=2)

print(results)

29/29 - 5s - loss: 0.4961 - accuracy: 0.8833
[0.4961329996585846, 0.883316159248352]


In [49]:
for i, j  in test_data_df.batch(15000):
  y = j
  y_pred = tf.where(model.predict(i) > 0.5, 1, 0)
  print(tf.math.confusion_matrix(labels=y, predictions=y_pred, num_classes=2))

tf.Tensor(
[[4142  696]
 [1000 8697]], shape=(2, 2), dtype=int32)


In [50]:
"""
slightly imballanced classification 1000 errors against 8697 correct predictions...
"""

'\nslightly imballanced classification 1000 errors against 8697 correct predictions...\n'